In [11]:
# load data, standardize , split training and testing
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

X = pd.read_csv('./data/nba/X_train.csv')
Y = pd.read_csv('./data/nba/y_train.csv')
sc = StandardScaler()
sc.fit(X)

X_train_std = sc.transform(X)

X_t = pd.read_csv('./data/nba/X_test.csv')
sc.fit(X_t)
X_test = sc.transform(X_t)

print(X.shape)
print(X.columns)


X_train, X_valid, y_train, y_valid = train_test_split(X_train_std, Y, test_size=0.3, random_state=0)


(52399, 8)
Index(['PERIOD', 'GAME_CLOCK', 'SHOT_CLOCK', 'DRIBBLES', 'TOUCH_TIME',
       'SHOT_DIST', 'PTS_TYPE', 'CLOSE_DEF_DIST'],
      dtype='object')


In [35]:
#Attempt to eliminate the outlier using randsac
from sklearn.linear_model import RANSACRegressor
ransac = RANSACRegressor(LinearRegression(), 
            max_trials=100, 
            min_samples=4000, 
            residual_threshold=0.7, 
            random_state=1)
s = [2,5,7]
ransac.fit(X_train[:,s], y_train)
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

X_train_masked = X_train[inlier_mask]
y_train_masked = y_train[inlier_mask]

print('Train samples eliminated:',sum(outlier_mask),'/',len(X_train))

ransac.fit(X_valid, y_valid)
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)
X_valid_masked = X_valid[inlier_mask]
y_valid_masked = y_valid[inlier_mask]
X_train_selected = X_train_masked[:,s]
X_valid_selected = X_valid_masked[:,s]

print('Valid samples eliminated:',sum(outlier_mask),'/',len(X_valid))

Train samples eliminated: 405 / 36679
Valid samples eliminated: 266 / 15720


In [38]:
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# clf = svm.LinearSVC(C=0.01)

# clf.fit(X_train_selected, y_train_masked)
# y_pred = clf.predict(X_train_selected)

gbm0 = GradientBoostingClassifier(random_state=10)
gbm0.fit(X_train_selected, y_train_masked)
y_pred = gbm0.predict(X_train_selected)
print(y_pred)
# y_predprob = gbm0.predict_proba(X)[:,1 ]
print('Train Accuracy: %.3f' % accuracy_score(y_pred, y_train_masked))

# X_valid_selected = X_valid_masked[:,idxs[0]]
# X_trans = poly.fit_transform(X_valid_selected)

# y_pred = clf.predict(X_valid_selected)
# print('Valid Accuracy: %.3f' % accuracy_score(y_pred, y_valid_masked))

C:\Users\a1989\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1 0 0 ... 1 1 0]
Train Accuracy: 0.628
